# Projet 2 - Analysez des données de systèmes éducatifs

## Table des matières
* [A - Analyse et reformulation de l'énoncé](#A)
* [B - Import et compréhension des données](#B)
    * [Description des données](#description)
* [C - Choix des indicateurs](#indicators)
* [D - Pré-filtrage des données](#prefilter)
    * [Suppression des autres indicateurs](#removing)
    * [Suppression des colonnes entièrement vides](#dropna)
    * [Suppression des pays trop petits](#small-countries)
    * [Suppression des années mal renseignées](#years)
    * [Taux de remplissage global de chaque indicateur](#fillings)
    * [Taux de remplissage par région géographique](#geofillings)
* [E - Regroupement des indicateurs](#grouping)
* [F - Second filtrage des données](#filtering)
    * [Suppression des pays trop pauvres](#poor-countries)
    * [Sélection du bon indicateur de population](#population)
    * [Remise au propre des données finales](#final-data)
* [G - Etude des indicateurs par région géographique](#geoanalysis)
* [H - Scoring des pays](#scoring)
    * [Classement des pays avec des coefficients égaux (référence)](#reference-ranking)
    * [Impact de la modification des pondérations - Méthode Borda](#borda)
* [I - Conclusion](#conclusion)

<a name="A"></a>
## A - Analyse et reformulation de l'énoncé
<ins>**Données issues de l'énoncé**</ins> : 
* Domaine de la **EdTech** pour Educational Technology (technologies de l'éducation).
* Academy est une **start-up** qui fournit des formation **en ligne** pour des niveaux **lycée et université**.
* Projet d'expansion à l'**international**.
* Les données sont fournies par la banque mondiale rubrique "[EdStats](https://datatopics.worldbank.org/education/)"

<ins>**Problématique**</ins> : analyser les [données](https://datacatalog.worldbank.org/dataset/education-statistics) de la banque mondiale afin de déterminer quels sont les pays avec les plus forts pontentiels **actuel et futur** d'expansion de Academy et déterminer **dans quels pays opérer en priorité**.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import re
import math
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

plt.style.use('seaborn-whitegrid')

<a name="B"></a>
## B - Import et compréhension des données
### Import des données

In [ ]:
# Data about countries
countries = pd.read_csv('../input/global-education-statistics/EdStatsCountry.csv')

# Data about each indicator (unit, date, ...)
indicators = pd.read_csv('../input/global-education-statistics/EdStatsSeries.csv')

# Data containing values of each indicators for each country and year
initial_data = pd.read_csv('../input/global-education-statistics/EdStatsData.csv')
# Creating a copy to work on
data = initial_data.copy(deep=True)

# We load the 2 other files but we don't use them
series = pd.read_csv('../input/global-education-statistics/EdStatsCountry-Series.csv', delimiter=',', doublequote=True)
footnotes = pd.read_csv('../input/global-education-statistics/EdStatsFootNote.csv')

### Paramètres globaux
Cette section regroupe les paramètres qui influent directement sur la façon dont sont pré-filtrées et filtrées les données.

In [ ]:
# Minimum population of the countries we keep in the dataset
MINIMUM_POPULATION = 2000000

# Maximum difference between the country revenue and the revenue of the reference country (France) - In percents (%)
MAX_REVENUE_PERCENT_DIFF = 20

# Minimum year to keep in the dataset (has been choosen by analysing the data below)
MIN_YEAR = 2005
MAX_YEAR = 2015

# Reference country code
REF_COUNTRY_CODE = "FRA"

<a name="description"></a>
### Description des données
Pour chacun des DataFrame ci-dessus j'utilise la méthode `.info()` qui nous permet de lister le nombre de lignes, le nombre de colonne, et pour chaque colonne, le nombre de valeur nulles.

In [ ]:
def format_percentage(value):
    '''
    Format a percentage with 1 digit after comma 
    '''
    return "{0:.1f}%".format(value * 100)


# We create a table containing all information about the 5 files
files_description = pd.DataFrame(columns = ["Nb lignes", "Nb colonnes", "Taux remplissage moyen", "Doublons", "Description"],
                                 index = ["EdStatsCountry.csv", 
                                          "EdStatsSeries.csv", 
                                          "EdStatsData.csv", 
                                          "EdStatsCountry-Series.csv", 
                                          "EdStatsFootNote.csv"])

# Filling the total number rows in each file
files_description["Nb lignes"] = [
    len(countries.index),
    len(indicators.index),
    len(initial_data.index),
    len(series.index),
    len(footnotes.index)
]

# Filling the number of columns in each file
files_description["Nb colonnes"] = [
    len(countries.columns),
    len(indicators.columns),
    len(initial_data.columns),
    len(series.columns),
    len(footnotes.columns)
]

# Filling the fill-percentile of each file
# We use the mean() function twice to calculate the mean for each columns, and then the mean for the whole file
files_description["Taux remplissage moyen"] = [
    format_percentage(countries.notna().mean().mean()),
    format_percentage(indicators.notna().mean().mean()),
    format_percentage(initial_data.notna().mean().mean()),
    format_percentage(series.notna().mean().mean()),
    format_percentage(footnotes.notna().mean().mean())
]

# FIlling the number of duplicate keys for each file
files_description["Doublons"] = [
    countries.duplicated(subset=["Country Code"]).sum(),
    indicators.duplicated(subset=["Indicator Name"]).sum(),
    initial_data.duplicated(subset=["Country Code", "Indicator Name"]).sum(),
    series.duplicated(subset=["CountryCode", "SeriesCode"]).sum(),
    footnotes.duplicated(subset=["CountryCode", "SeriesCode", "Year"]).sum(),
]

# Finally adding a short description for each file
files_description["Description"] = [
    "Liste des pays avec leurs données principales",
    "Liste des indicateurs avec description, unité, période, etc...",
    "Données de chaque indicateur par pays et par année",
    "Description des différentes séries de données (majoritairement provenance)",
    "Commentaire pour chaque couple série de données / pays"
]


files_description

<a name="indicators"></a>
## C - Choix des indicateurs
L'idée est maintenant de trouver les indicateurs qui nous intéressent parmi les 3665 proposés. Pour répondre à notre problématique, on va tout d'abord se focaliser sur les idées suivantes : 
* Définir la **quantité d'étudiants** par pays en se basant, par exemple, sur la population par tranche d'âge.
* Définir l'**accès à internet**  : accès des étudiants à un ordinateur et une connexion internet correcte.
* Définir le **revenu moyen** du pays afin de savoir si les étudiants auront de quoi payer les cours en ligne.
* Définir la **langue principale du pays** : le français et l'anglais étant plus facile à traduire.

J'utilise la [liste des indicateurs](http://datatopics.worldbank.org/education/) afin de les trouver plus facilement.


In [ ]:
# Preparing a list that will contains all the indicator codes we want to keep
choosen_indicators = []

### Population étudiante
Plusieurs indicateurs peuvent nous aider à cibler la population qui nous intéresse, sachant que l'on cherche à chiffrer l'effectif de la population lycée/université, soit environ la tranche d'âge 15-25 ans.
* **SP.POP.TOTL** : population totale à la mi-année décomposée en **SP.POP.TOTL.FE.IN** (femmes) et **SP.POP.TOTL.MA.IN** (hommes)
* **SP.POP.GROW** : taux de croissance de la population -> utile pour les projections futures
* **SP.POP.1524.TO.UN** : population totale de la tranche 15-24 ans
* **SP.SEC.TOTL.IN** : population totale ayant l'âge d'entrer en éducation secondaire (secondary ed.)
* **SP.SEC.UTOT.IN** : population totale ayant l'âge d'entrer en éducation secondaire élevée (upper secondary ed.)
* **SP.TER.TOTL.IN** : population totale ayant l'âge d'entrer en éducation tertiaire (tertiary ed.)

On va **tous les conserver dans un premier temps** et on analysera ensuite lequel (ou lesquels) sont les plus pertinents et les plus fiables pour notre utilisation (voir [plus bas dans le fichier](#population)).

In [ ]:
population_indicators = ["SP.POP.TOTL", "SP.POP.GROW", "SP.POP.1524.TO.UN", "SP.SEC.TOTL.IN", "SP.SEC.UTOT.IN", 
                         "SP.TER.TOTL.IN"]

### Accès à internet
En terme de communication, deux indicateurs nous indiquent le pourcentage d'ordinateurs personnels et de connexion internet pour 100 habitants :

* **IT.NET.USER.P2** : pourcentage d'utilisateurs d'internet
* **IT.CMP.PCMP.P2** : pourcentage de possesseur d'un ordinateur personnel

J'ai également recherché des indicateurs concernant les **compétences informatique** mais si plusieurs sont indiqués dans la documentation de la banque mondiale, aucun n'est présent dans le dataset.

In [ ]:
internet_indicators = ["IT.NET.USER.P2", "IT.CMP.PCMP.P2"]

### Revenu moyen du pays
Même si cet indicateur sera probablement fortement lié à l'accès à un ordinateur personnel, il nous permettra de savoir si la population a, ou non, sensiblement le même pouvoir d'achat que notre pays de référence.

Le dataset nous donne accès au **PNB par habitant** ainsi qu'au **PIB par habitant**. Ce dernier est le plus représentatif du niveau de vie moyen du pays, surtout qu'il est ici exprimé en "Parité de Pouvoir d'Achat" (**PPA** ou en anglais PPP) ce qui signifie qu'on tient compte du coût de la vie dans le pays. Cela permet d'avoir une comparaison plus fiable entre les pays.
* **NY.GDP.MKTP.PP.CD** :  PIB (PPP - Current US Dollar)
* **NY.GDP.PCAP.PP.CD** : PIB par habitant (PPP - Current US Dollar)

Pour nous, cet indicateur sera **obligatoire** : on supprimera du dataset tous les pays pour lesquels il n'est pas renseigné.

In [ ]:
revenue_indicators = ["NY.GDP.PCAP.PP.CD"]

### Langue principale
Après étude détaillée des indicateurs, aucun ne semble actuellement nous renseigner sur la langue principale parlée dans le pays. C'est éventuellement une donnée que nous pourrons intégrer à posteriori pour départager 2 pays très proches en termes de score.

### Récapitulatif des indicateurs

In [ ]:
choosen_indicators.extend(population_indicators)
choosen_indicators.extend(internet_indicators)
choosen_indicators.extend(revenue_indicators)
choosen_indicators

<a name="prefilter"></a>
## D - Pré-filtrage des données
Le pré-filtrage effectué ici est une succession d'opération de filtrages facile à appliquer. Le but est de venir amputer le Dataset d'un grand nombre de lignes afin de faciliter les opérations de filtrages plus minutieuses à venir ensuite.
<a name="removing"></a>
### Suppression des indicateurs non retenus
On ne conserve maintenant que les lignes des indicateurs qui nous intéressent et on supprime également les colonnes non utiles tel que les noms de pays et les noms des indicateurs.

In [ ]:
# Filtering the data set on the selected indicators
data = data[data["Indicator Code"].isin(choosen_indicators)]

# Displaying a warning if one choosen indicator is not in the dataset
if len(choosen_indicators) > len(data["Indicator Code"].unique()):
    print("ATTENTION : certains indicateurs n'ont pas été trouvé dans le tableau de données")
    
# Also removing columns that are not useful for the moment (we keep the keys columns)
data = data.drop(["Country Name", "Indicator Name"], axis=1, errors="ignore")

data

<a name="dropna"></a>
### Suppression des colonnes entièrement vides

In [ ]:
data.dropna(axis=1, how="all", inplace=True)

<a name="small-countries"></a>
### Suppression des pays trop petits
Afin de faciliter le travail d'analyse à venir et réduire significativement le nombre de ligne, nous allons commencer par éliminer les pays à la population trop faible.

**Rappel** : la population minimale retenue ici est indiquée en début de Notebook au niveau des paramètres.

In [ ]:
# Step 0 - Retrieving the population data from the main dataset
countries_populations = data[data["Indicator Code"] == "SP.POP.TOTL"]

# Step 1 - Removing columns where all values are NaN
countries_populations.reset_index(inplace=True)
countries_populations.set_index("Country Code", inplace=True)

# Step 2 - Calculating the mean population for each country
countries_populations = countries_populations.mean(axis=1).to_frame()
countries_populations.columns = ["Population"]

# Step 3 - Keeping countries codes
biggest_countries_codes = countries_populations[countries_populations["Population"] >= MINIMUM_POPULATION].index.values

# Finally filtering data
data = data[data["Country Code"].isin(biggest_countries_codes)]

<a name="years"></a>
### Suppression des années trop anciennes et mal renseignées
On va maintenant analyser le taux de remplissage des données de nos indicateurs retenus, par année, ce qui nous permettra d'en supprimer une partie.

In [ ]:
# For the next filterings, want to know the global "filling percentage" of all selected indicators merged
year_filling = data.drop(columns=["Country Code", "Indicator Code"]).notnull().sum(axis=0).to_frame()

# We calculate it as a percentile (max possible values per year = nb of countries * nb of indicators)
max_values = len(choosen_indicators) * len(data["Country Code"].unique())
year_filling = 100 * year_filling / max_values

fig1 = plt.figure(figsize=(15,5))
plt.title("Taux de remplissage par année (tous pays confondus, sur les indicateurs sélectionnés)")
plt.xticks(rotation = 90)
plt.ylim(ymin=0, ymax=100)
plt.ylabel("Taux de remplissage (%)")
plt.plot(year_filling.index.values, year_filling[0], color="dodgerblue", linewidth=3)

On constate plusieurs choses : 
* Les années **après 2016** avaient déjà été supprimées, ce qui signifie qu'elles étaient vides pour nos indicateurs. ON va toutefois exclure en plus l'année 2016 qui est significativement moins renseignée que 2015.
* Les années **avant 1990** sont moins remplies, et nous intéressent peu car internet n'est apparu qu'en 1991.

De plus, au vu de l'évolution rapide des nouvelles technologies, **il parait peut pertinent de conserver les années antérieures à 2005** (apparition de Youtube, Facebook, essor des réseaux sociaux et des plateformes de streaming) car elles sont peu représentatives des nouvelles technologies, et risqueraient ainsi de faire chuter nos indicateurs "technologiques" (accès internet et accès à des ordinateurs).

**CONCLUSION** : on ne conserve que **les années après 2005 (inclue) et avant 2016 (exclue)**.

Ces 2 années sont mises en paramètre en début de Notebook.

In [ ]:
# The MIN_YEAR variable has been defined at the beginning of the notebook

# We save the data before filtering it because we need it below
data_before_year_filtering = data.copy(deep=True)

# Isolating the columns we want to drop from the DF
years = list(filter(lambda x: re.match("^\d{4}$", x), data.columns.values))
dropped_years = [year for year in years if int(year) < MIN_YEAR or int(year) > MAX_YEAR]

# Finally removing the unwanted values
data = data.drop(columns=dropped_years)

data

<a name="fillings"></a>
### Taux de remplissage global de chaque indicateur
On vérifie maintenant les taux de remplissage de nos indicateurs sur les années sélectionnées.

In [ ]:
# We count how many values (non NaN) we have for each indicator and we devide it by the number of countries
indicator_filling = (data
                     .drop(columns=["Country Code"])
                     .set_index("Indicator Code")
                     .notnull()
                     .groupby("Indicator Code")
                     .sum())
indicator_filling = 100 * indicator_filling / len(data["Country Code"].unique())

# Plotting the result
fig2 = plt.figure(figsize=(15,5))
plt.title("Taux de remplissage global de chacun des indicateurs sélectionnés")
plt.xticks(rotation = 90)
plt.ylabel("Taux de remplissage (%)")
plt.bar(indicator_filling.index.values, indicator_filling.mean(axis=1))

On constate que tous les indicateurs sont **très bien renseignés** à l'exception de "**IT.CMP.PCMP.P2**".

Cet indicateur est à 0% de remplissage sur les années sélectionnées, ce qui est gênant car c'est un indicateur primordial pour la suite de notre étude. 

Nous allons voir à quel point il est rempli dans les autres années.

In [ ]:
# We calculate the filling percentage of the given indicator for each year > 0
pcmp_filling = data_before_year_filtering[data_before_year_filtering["Indicator Code"] == "IT.CMP.PCMP.P2"]
pcmp_filling = (pcmp_filling
                .drop(columns=["Country Code"])
                .set_index("Indicator Code")
                .notnull()
                .groupby("Indicator Code")
                .sum())
pcmp_filling = pcmp_filling / len(data_before_year_filtering["Country Code"].unique())
pcmp_filling = pcmp_filling.loc[:, (pcmp_filling != 0).any(axis=0)]

# We plot the results
fig3 = plt.figure(figsize=(15,5))
plt.title("Taux de remplissage de l'indicateur IT.CMP.PCMP.P2 par année")
plt.xticks(rotation = 90)
plt.ylim(ymin=0, ymax=100)
plt.plot(pcmp_filling.columns.values, 100 * pcmp_filling.loc["IT.CMP.PCMP.P2"], color="orange", linewidth=3)

**CONCLUSION** : pour cet indicateur-là, nous allons prendre une moyenne sur les années **2001-2005**.

<a name="geofillings"></a>
### Taux de remplissage par région géographique
On veut maintenant regarder si les indicateurs sont mieux remplis dans certaines portions du monde, ce qui nous permettrait d'éliminer certains continent si les indicateurs y sont très mal renseignés.

Pour cela, on utilise la colonne "Region" du fichier des pays ; et on calcul un pourcentage au prorata du nombre de pays dans chaque continent (pour éviter d'avoir, par exemple, l'Amérique du Noed avec un score faible, du fait qu'elle ne contient que 2 pays).

In [ ]:
# Step 1 - We isolate country code and region from country file and kipping only on our selected countries
regions = (countries[countries["Country Code"].isin(biggest_countries_codes)]
           .set_index("Country Code")
           .filter(items=["Region"]))

# Step 2 - We also calculate how many countries we have per region in order to calculate our percentile 
country_per_region = (regions
                      .reset_index()
                      .set_index("Region")
                      .notnull()
                      .groupby("Region")
                      .sum())
country_per_region.columns = ["Total countries"]

# Step 3 - We count how many indicators are filled for each Region
geographical_data = data.copy(deep=True)
geographical_data = (geographical_data
                     .merge(regions, how="left", on="Country Code")
                     .drop(columns=["Country Code", "Indicator Code"])
                     .dropna(axis=0, how="any", subset=["Region"])
                     .set_index("Region")
                     .notnull()
                     .groupby("Region")
                     .sum())
nb_of_years = len(geographical_data.columns)
print(geographical_data)
geographical_data = geographical_data.sum(axis=1).to_frame().astype("float")


# Step 4 - We divide the number of filled indicators by the number of theoretical indicators per region
for region, total in geographical_data.itertuples():
    region_max = country_per_region.loc[region]["Total countries"] * nb_of_years * len(choosen_indicators)
    geographical_data.loc[region][0] = geographical_data.loc[region][0] / region_max

# Step 5 - We sort the results to improve chart readability
geographical_data.sort_values(by=[0], axis=0, ascending=False, inplace=True)

# We plot the results
fig4 = plt.figure(figsize=(15,5))
plt.title("Taux de remplissage global par zone géographique")
plt.xticks(rotation = 90)
plt.bar(geographical_data.index.values, 100 * geographical_data[0], color="chocolate")

**CONCLUSION** : pour les indicateurs sélectionnés, on ne note pas de disparité particulière en terme de remplissage. **Toutes les zones géographiques peuvent donc être analysées.**

<a name="grouping"></a>
## E - Regroupement des indicateurs
Le but de cette partie est de créer un tableau qui contient une ligne par pays, et une colonne par indicateur avec la valeur finale conservée pour celui-ci. On va traiter séparément notre indicateur **IT.CMP.PCMP.P2** qui n'a pas de valeur pour les années sélectionnées.

In [ ]:
# We create a table with the mean value of each indicator for each country
final_indicators_values = (
                           (data[data["Indicator Code"].isin(choosen_indicators)]
                            .groupby(["Country Code", "Indicator Code"])
                            .mean()
                           )
                           .mean(axis=1)
                           .to_frame()
                           .unstack(level=1)
                           .droplevel(0, axis=1)
                           .drop(columns=["IT.CMP.PCMP.P2"]))

# Now we have to fill the IT.CMP.PCMP.P2 indicator with the 2011-2005 years
pcmp_data = initial_data.copy(deep=True)
kept_years = ["2001", "2002", "2003", "2004", "2005"]
pcmp_data = (pcmp_data[(pcmp_data["Indicator Code"] == "IT.CMP.PCMP.P2") 
                       & (pcmp_data["Country Code"].isin(biggest_countries_codes))]
             .drop(columns=["Country Name", "Indicator Name", "Indicator Code"])
             .set_index("Country Code")
             .filter(items=kept_years)
             .mean(axis=1)
             .to_frame())
pcmp_data.columns = ["IT.CMP.PCMP.P2"]

# And we merge it with the previous dataframe
final_indicators_values = final_indicators_values.merge(pcmp_data, how="left", left_index=True, right_index=True)

final_indicators_values

<a name="filtering"></a>
## F - Second filtrage des données
<a name="poor-countries"></a>
### Suppression des pays trop pauvres
Nous allons à présent enlever du dataset les pays dont le PIB est trop inférieur à celui **de la France** qui est notre pays de référence.

Nous pouvons nous appuyer sur le **PNB par habitant PPA** car nous avons vérifé (lors de l'étape suivante) que c'était un indicateur fiable dans notre set d'années et de pays.

In [ ]:
# At first we calculate the mean GDP PPP of our reference country (France)
fra_gdp_ppp = (data[(data["Country Code"] == "FRA") & (data["Indicator Code"] == "NY.GDP.PCAP.PP.CD")]
               .drop(columns=["Country Code", "Indicator Code"])
               .mean(axis=1)
               .values[0])

# Now we calculate the min GDP PPP we want 
min_gdp_ppp = fra_gdp_ppp * (1 - (MAX_REVENUE_PERCENT_DIFF / 100))

final_indicators_values = final_indicators_values[final_indicators_values["NY.GDP.PCAP.PP.CD"] >= min_gdp_ppp]

print("Après filtrage par PIB, il reste " + str(len(final_indicators_values)) + " pays.")
final_indicators_values

<a name="population"></a>
### Sélection du bon indicateur de population
Il nous reste à présent à choisir l'indicateur de population le plus efficace. Le plus pratique serait le nombre d'habitants de la tranche 15-24 ans mais on avait vu plus haut qu'il n'est pas très bien renseigné. Regardons à présent à quel point il est fiable sur les derniers pays restants.

Pour cela on va afficher la liste des pays pour lesquels il n'est pas renseigné et regarder si ces pays sont importants pour nous ou non.

In [ ]:
pop_1524 = (final_indicators_values.merge(countries[["Country Code", "Short Name"]].set_index("Country Code"), 
                              left_index=True, 
                              right_index=True))[["SP.POP.1524.TO.UN", "Short Name"]]

pop_1524[pop_1524["SP.POP.1524.TO.UN"].isnull()]

On constate que, à part Puerto Rico, les autres lignes ne sont pas des pays mais des regroupement de pays. 
Dans la suite de cette analyse, on fait donc les choix suivants : 
* Ignorer Puerto Rico
* Ne conserver comme indicateur de population que cet indicateur **SP.POP.1524.TO.UN**

<a name="final-data"></a>
### Remise au propre des données finales
On peut maintenant conserver uniquement nos 5 indicateurs finaux et remettre ainsi nos données au propre en ajoutant au passage le nom de chaque pays afin de mieux s'y retrouver.

In [ ]:
indicators_names = {
    'IT.NET.USER.P2': 'Utilisateurs Internet (%)', 
    'NY.GDP.PCAP.PP.CD': 'PIB par hab (PPA)', 
    'SP.POP.1524.TO.UN': 'Population 15-24 ans',
    'SP.POP.GROW': 'Croissance pop. (%)',
    'IT.CMP.PCMP.P2': 'Possesseurs PC (%)',
    'Short Name': 'Country Name'
}

final_indicators_values = (final_indicators_values
                           .dropna(subset=["SP.POP.1524.TO.UN"])
                           .drop(columns=["SP.POP.TOTL", "SP.SEC.TOTL.IN", "SP.SEC.UTOT.IN", "SP.TER.TOTL.IN"], errors="ignore")
                           .merge(countries[["Country Code", "Short Name"]].set_index("Country Code"), 
                              left_index=True, 
                              right_index=True)
                           .rename(columns=indicators_names))

final_indicators = indicators_names.keys()
final_indicators_values

<a name="geoanalysis"></a>
## G - Etude des indicateurs par région géographique
Maintenant que nous avons sélectionné nos 5 indicateurs principaux, nous allons étudier leurs indicateurs statistiques pour les différentes zones géographiques.


In [ ]:
indicator_analysis_base_data = initial_data.copy(deep=True)

indicator_analysis_base_data = (indicator_analysis_base_data[indicator_analysis_base_data["Indicator Code"].isin(final_indicators)]
                           .drop(columns=["Country Name", "Indicator Name"])
                           .reset_index())

# We add the geographic region
regions = (countries[countries["Country Code"].isin(biggest_countries_codes)]
           .set_index("Country Code")
           .filter(items=["Region"]))
indicator_analysis_base_data = indicator_analysis_base_data.merge(regions, left_on="Country Code", right_index=True)

### Indicateurs principaux : moyenne, médiane, écart-type

In [ ]:
# We use the initial data for that 
indicator_analysis = indicator_analysis_base_data.copy(deep=True)

indicator_analysis = (indicator_analysis
                           .drop(columns=["index", "Country Code"], errors="ignore")
                           .groupby(["Indicator Code", "Region"]).mean())

# Calculating the main statistic indicators
indicator_analysis_mean = indicator_analysis.mean(axis=1).to_frame()
indicator_analysis_std = indicator_analysis.std(axis=1).to_frame()
indicator_analysis_med = indicator_analysis.median(axis=1).to_frame()

indicator_analysis_mean = indicator_analysis_mean.merge(indicator_analysis_std, left_index=True, right_index=True)
indicator_analysis_mean = indicator_analysis_mean.merge(indicator_analysis_med, left_index=True, right_index=True)

indicator_analysis_mean.columns = ["Mean", "Std", "Median"]

indicator_analysis_mean

### Evolution des indicateurs choisis

In [ ]:
indicator_analysis_data = indicator_analysis_base_data.copy(deep=True)

# Calculating means
indicator_analysis_data = (indicator_analysis_data
                           .drop(columns=["Country Code"], errors="ignore")
                           .groupby(["Region", "Indicator Code"])
                           .mean()).reset_index().drop(columns=["index"])

colors = ["#d43737", "#d48b37", "#cfd437", "#42d435", "#35d4c9", "#3848d6", "#c22ed9"]
legend = {}

# Preparing the plot
x_ticks = []
x_ticks_labels = []
for column_name in indicator_analysis_data.columns:
    if re.match('r/^\d{4}$/', column_name):
        x_ticks.append(int(year))
        if year.endswith('0'):
            x_ticks_labels.append(str(year))
        else:
            x_ticks_labels.append("")
        
indicators = indicator_analysis_data["Indicator Code"].unique()
fig10, axes = plt.subplots(nrows=math.ceil(len(indicators)/2), ncols=2, figsize=(15,5*len(indicators)/2))
i = 0
for indicator in indicators:
    subdata = indicator_analysis_data[indicator_analysis_data["Indicator Code"] == indicator]\
                .dropna(how="any", axis=1)\
                .drop(columns=["Indicator Code"])\
                .set_index("Region")
    row_number = 0
    for row in subdata.iterrows():
        axes[math.floor(i/2), i%2].title.set_text(indicators_names[indicator])
        axes[math.floor(i/2), i%2].set_xticks(x_ticks)
        axes[math.floor(i/2), i%2].set_xticklabels(x_ticks_labels)
        axes[math.floor(i/2), i%2].plot(subdata.columns, subdata.iloc[row_number].values, color=colors[row_number])    
        
        # Storing the association serie/color for the legend
        legend[subdata.index[row_number]] = colors[row_number]
        row_number += 1
    i+=1

    
    
fig10.legend(legend.keys(), labelcolor=legend.values(), loc="upper center")


<a name="scoring"></a>
## H - Scoring des pays
La stratégie de scoring est la suivante : 
* On divise chaque valeur **par le maximum de sa colonne** de façon à avoir des valeurs entre 0 et 1.
* On va ensuite calculer la **somme pondérée** des colonnes pour avoir le score du pays.

Concernant les pondérations : 
* Dans un premier temps on applique une **pondération similaire** à tous les indicateurs,
* Ensuite on fera **varier les pondération** afin de voir si le classement change de manière significative.



In [ ]:
def calculate_scores(coefficients, plot=False, score_name=None, return_index=False):
    """ Plot a bar chart with the scores calculated using the given coefficients
    
    Parameters:
    coefficient (dict): dictionnary of column_name: coefficient
    
    Returns:
    DataFrame: the scores sorted by DESC order or the list of sorted index
    
    """
    
    # We work on a copy of the data 
    scoring_data = final_indicators_values.copy(deep=True)
    score_column_name = "Score"

    # Dividing all columns by its max value to have [0;1] values
    scoring_data.iloc[:,:-1] = scoring_data.iloc[:,:-1].apply(lambda x: x / x.max())
    
    # For each row of the dataframe, we calculate the score
    for country_code, row in scoring_data.iterrows():
        score = 0
        for column, coef in coefficients.items():
            score += row[column] * coef
        scoring_data.at[country_code, score_column_name] = score

   
    # Dividing by the reference country score to easily get the highest potential countries
    ref_score = scoring_data.loc[REF_COUNTRY_CODE][score_column_name]
    scoring_data[score_column_name] = scoring_data[score_column_name] / ref_score

     # Sorting the results by score desc    
    scoring_data = scoring_data.sort_values(score_column_name, ascending=False)
    subset = scoring_data[score_column_name].to_frame()
    
    # Renaming the score column if asked
    if score_name != None:
        subset = subset.rename(columns={score_column_name: score_name})
        score_column_name = score_name

    # Returning the result and plotting the result if asked
    if plot:
        plt.xticks(rotation = 90)
        plt.bar(scoring_data["Country Name"], subset[score_column_name], color="goldenrod", edgecolor="#333333")

    if return_index:
        subset = subset.astype("string")
        for country_code, row in subset.iterrows():
            subset.at[country_code, score_column_name] = country_code
        subset = subset.reset_index().drop(columns=["Country Code"])
        
    return subset

<a name="reference-ranking"></a>
### Classement des pays avec des coefficients égaux (référence)

In [ ]:
fig8 = plt.figure(figsize=(15,7))
plt.title("Classement de référence")
plt.ylabel("Score du pays")

scores = calculate_scores({
    'Utilisateurs Internet (%)': 1,
    'PIB par hab (PPA)': 1,
    'Population 15-24 ans': 1,
    'Croissance pop. (%)': 1,
    'Possesseurs PC (%)': 1
}, plot=True)

<a name="borda"></a>
### Impact de la modification des pondérations - Méthode Borda
On va maintenant regarder si notre classement est sensible ou non à une modification des pondérations.
Pour cela on va utiliser la méthode **Borda** : 
* On va appliquer un coefficient de 3 à un indicateur, 2 à un autre, et 1 à tous les indicateurs restants
* Pour 5 indicateurs, on va ainsi obtenir 20 classements différents de nos pays
* On applique alors la méthode Borda en considérant qu'on veut le classement des 10 meilleurs pays

Ainsi, pour chaque classement, le premier pays recevra 10 points, le second 9 points, etc... A partir du 11e, ils reçoivent tous 0 points.

En sommant, pour chaque pays, ses points obtenus à chaque classement, on obtiendra un classement global.

In [ ]:
# Step 1 : we generate all possible indicator/weight possibilities
indicators = [
    'Utilisateurs Internet (%)',
    'PIB par hab (PPA)',
    'Population 15-24 ans',
    'Croissance pop. (%)',
    'Possesseurs PC (%)'
]
weights = [3,2]

all_coefficients = []
for combination in itertools.permutations(indicators, len(weights)):
    new_coefficients = {}
    for index, indicator in enumerate(indicators):
        if indicator in combination:
            new_coefficients[indicator] = weights[combination.index(indicator)]
        else: 
            new_coefficients[indicator] = 1
    all_coefficients.append(new_coefficients)
    
print("Les " + str(len(all_coefficients)) + " cas possibles ont été générés")

j=0
for coefficients in all_coefficients:
    print("\n")
    print("Classement #" + str(j) + " : ")
    for indic, coef in coefficients.items():
        if coef > 1:
            print(str(indic) + " : " + str(coef))
    
    j += 1

In [ ]:
# Step 2 : we generate all rakings
reference_coefficients = {
    'Utilisateurs Internet (%)': 1,
    'PIB par hab (PPA)': 1,
    'Population 15-24 ans': 1,
    'Croissance pop. (%)': 1,
    'Possesseurs PC (%)': 1
}

# We initialize a dataframe with all the coefficients to weight 1 
rankings = calculate_scores(reference_coefficients, score_name="Reference score", return_index=True)
top_10 = rankings["Reference score"].head(15).tolist()

for index, coefficient_case in enumerate(all_coefficients):
    weighted_scores = calculate_scores(coefficient_case, 
                                       score_name="CASE_" + str(index), 
                                       return_index=True)
    rankings = rankings.merge(weighted_scores, left_index=True, right_index=True)


# Coloring the dataframe to highlight the top 10 of the reference ranking and see the differences
rankings_colors = rankings.copy(deep=True)
rankings_colors = rankings_colors.isin(top_10)

def color_cells(value):
    if value == True:
        return 'text-align:center; width: 120px; background-color:#ffefe0; font-weight:bold; border:1px solid #fac28e;'
    else:
        return 'text-align:center; width: 120px; '

print("Les couleurs nous permettent de visualiser que le Top 10 de référence ne changent pas beaucoup")
rankings.style.apply(lambda x: rankings_colors.applymap(color_cells), axis=None)

Maintenant on part de ce tableau et on calcule le score de chaque pays : 

In [ ]:
scores = {}
MAX_SCORE = 15

for index, row in enumerate(rankings.values.tolist()):
    for country in row:
        if not country in scores:
            scores[country] = 0
        score = MAX_SCORE - index
        scores[country] += max([score, 0])

# Sorting by score DESC
scores = dict(sorted(scores.items(), key=lambda item: item[1], reverse=True))

scores

<a name="conclusion"></a>
## I - Conclusion
Il ne nous reste plus qu'à tracer les scores sur un graphiques : 

In [ ]:
# Renaming the country code by the country name and keeping only scores > 0
named_scores = {}
for country_code, score in scores.items():
    if score > 0:
        country_name = countries[countries["Country Code"] == country_code]["Short Name"].values[0]
        named_scores[country_name] = score

fig9 = plt.figure(figsize=(15,7))
plt.xticks(rotation = 90)
plt.title("Classement final des pays")
plt.ylabel("Score du pays")
plt.bar(named_scores.keys(), named_scores.values(), color="gold", edgecolor="black")